# Imports

In [19]:
import pandas as pd
import numpy as np

# Data Cleaning

## Selecting Relevant Fields
the dataset will be loaded and transformed and relevant dimensions will remain

In [20]:
#TODO: remove the nrows argument when done testing
df = pd.read_csv('database.csv', nrows=2500)[[
    'NAME_CONTRACT_STATUS',
    'CODE_GENDER',
    'FLAG_OWN_CAR',
    'FLAG_OWN_REALTY',
    'CNT_CHILDREN',
    'AMT_INCOME_TOTAL',
    'AMT_CREDIT',
    'NAME_INCOME_TYPE',
    'NAME_EDUCATION_TYPE',
    'NAME_FAMILY_STATUS',
    'NAME_HOUSING_TYPE',
    'DAYS_BIRTH',
    'DAYS_EMPLOYED',
    'OCCUPATION_TYPE',
    'CNT_FAM_MEMBERS'
]]
df.head()

,NAME_CONTRACT_STATUS,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,Approved,F,N,Y,0,171000.0,491580.0,State servant,Secondary / secondary special,Married,House / apartment,-14548,-1187,Medicine staff,2.0
1,Approved,F,Y,Y,1,175500.0,29700.0,State servant,Higher education,Married,House / apartment,-11081,-3244,High skill tech staff,3.0
2,Approved,F,N,Y,0,135000.0,48600.0,Working,Secondary / secondary special,Married,House / apartment,-12939,-629,Sales staff,2.0
3,Approved,M,N,N,0,180000.0,196740.0,Working,Secondary / secondary special,Married,Rented apartment,-8945,-672,Sales staff,2.0
4,Refused,M,Y,N,0,225000.0,774229.5,Pensioner,Secondary / secondary special,Married,House / apartment,-23919,365243,NaN,2.0


## Reductions to binary variables

The target variable, NAME_CONTRACT_STATUS, will be reduced from one of 4 possible values, to one of two generic but still correct values – for example, the dataset distingushes between cancelled and rejected and granted loans, however we will only distinguish between granted and not granted loans. Values that are binary but that don't use the binary alphabet will be transformed to use the binary alphabet as well.

Defining a function that will return a copy of the dataframe with reduced fields.

In [21]:
def reduce(df, name, value):
    if type(name) != str:
        raise Exception('only one dimension is reduced at a time')
    idx_name = df.columns.get_loc(name)
    reduced = [(1 if df[name][i] == value else 0) for i in range(len(df))]
    df_reduced = df.drop(labels=[name], axis=1)
    df_reduced.insert(loc=idx_name, column=name, value=reduced)
    return df_reduced

In [22]:
df0 = reduce(df, 'NAME_CONTRACT_STATUS', 'Approved')  # 1 if approved else 0
df1 = reduce(df0, 'CODE_GENDER', 'M')  # 1 if male else 0
df2 = reduce(df1, 'FLAG_OWN_CAR', 'Y')  # 1 if owns car else 0
df3 = reduce(df2, 'FLAG_OWN_REALTY', 'Y')  # 1 if owns property else 0

## One-Hot Encoding
defining a function that returns a copy of the input dataframe with a specific dimension one-hot encoded

In [23]:
def one_hot_encode(df, name):
    if type(name) != str:
        raise Exception('one hot encoding applies to one dimension at a time')
    if len(df) == 0:
        raise Exception('dataframe is empty')
    
    df = df.copy()
    values = df[name].unique()
    
    #for each unique value, we create a new column where df[row][new column] is 1 if the value of df[row][value] == new column
    for v in values:
        one_hot_column = [(1 if df[name][i] == v else 0) for i in range(len(df))]
        df.insert(loc=len(df.loc[0]), column=v, value=one_hot_column)

    return df.drop(labels=[name], axis=1)

performing one-hot encoding on any dimension whose values are one of a set of string values

In [24]:
df4 = one_hot_encode(df3, 'NAME_INCOME_TYPE')
df5 = one_hot_encode(df4, 'NAME_EDUCATION_TYPE')
df6 = one_hot_encode(df5, 'NAME_FAMILY_STATUS')
df7 = one_hot_encode(df6, 'NAME_HOUSING_TYPE')
df8 = one_hot_encode(df7, 'OCCUPATION_TYPE')
df8.head(20)

,NAME_CONTRACT_STATUS,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,DAYS_BIRTH,DAYS_EMPLOYED,CNT_FAM_MEMBERS,...,Accountants,Cleaning staff,Secretaries,Private service staff,Waiters/barmen staff,Low-skill Laborers,Security staff,HR staff,Realty agents,IT staff
0,1,0,0,1,0,171000.0,491580.0,-14548,-1187,2.0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,1,1,175500.0,29700.0,-11081,-3244,3.0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,135000.0,48600.0,-12939,-629,2.0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,180000.0,196740.0,-8945,-672,2.0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,1,0,0,225000.0,774229.5,-23919,365243,2.0,...,0,0,0,0,0,0,0,0,0,0
5,0,1,1,0,1,225000.0,36166.5,-15173,-3397,3.0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,1,1,90000.0,120582.0,-18834,365243,3.0,...,0,0,0,0,0,0,0,0,0,0
7,1,1,0,0,0,135000.0,30550.5,-9950,-146,2.0,...,0,0,0,0,0,0,0,0,0,0
8,0,1,1,0,1,180000.0,0.0,-12686,-4410,3.0,...,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,54000.0,112500.0,-23154,365243,1.0,...,0,0,0,0,0,0,0,0,0,0


# Training the models

## Split the dataframe into X and y as numpy arrays

convert the dataframe into a numpy tensor

In [25]:
dataset_as_array = np.array(df8)
X = dataset_as_array[:,1:]
y = dataset_as_array[:,0].reshape((-1,1)).astype(np.int32)

normalize the data

In [27]:
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit_transform(X)
print(X)

58870251e-01  6.15572864e-01 -5.49004115e-01 -1.82988828e-01
  -1.04488820e-01  7.30576711e-01 -2.60614068e-01 -3.79000467e-01
  -2.50848899e-01 -3.32592098e-01 -2.94786745e+00 -1.17420229e-01
   5.20479468e+00 -7.50435647e-02 -2.17577014e-01 -5.66592700e-02
  -1.65945831e-01 -1.94357944e-01 -3.63656949e-01  0.00000000e+00
  -2.45398708e-01 -4.63434594e-01 -1.49982957e-01 -3.19072035e-01
  -2.55322548e-01 -1.65945831e-01 -1.49982957e-01 -5.66592700e-02
  -1.10207754e-01 -8.02572354e-02 -8.75111555e-02 -1.30717492e-01
  -3.46618195e-02 -5.29892632e-02 -3.46618195e-02]
 [-6.88517686e-01 -7.13267660e-01  6.18044492e-01 -5.61770525e-01
  -6.93583493e-01 -6.21399096e-01 -7.47695376e-01 -4.90231088e-01
  -1.65914362e-01 -2.63229921e-01  9.82552189e-01 -4.81802923e-01
  -5.58870251e-01  6.15572864e-01 -5.49004115e-01 -1.82988828e-01
  -1.04488820e-01  7.30576711e-01 -2.60614068e-01 -3.79000467e-01
  -2.50848899e-01 -3.32592098e-01  3.39228279e-01 -1.17420229e-01
  -1.92130538e-01 -7.50435647e

## Creating the test class
We will create a class that collects everything we need to build and to analyze a model. The class will point to our data, and to a model building algorithm, and will provide functionality for presenting results.

In [15]:
from sklearn.model_selection import train_test_split

class Test:
    def __init__(self, name, X, y, algorithm, args):
        self.name = name
        self.X = X  #pointer, not copy
        self.y = y
        self.algorithm = algorithm
        self.args = args
    
    def run(self, split=0.8):
        X_train, X_test = train_test_split(self.X, train_size=split, test_size=(1.0-split))
        y_train, y_test = train_test_split(self.y, train_size=split, test_size=(1.0-split))
        model = self.algorithm(**self.args)
        print("before")
        model.fit(X_train, y_train.ravel())
        print("after")
        acc_train = model.score(X_train,y_train.ravel())
        acc_test = model.score(X_test,y_test)
        return acc_train, acc_test

## Support Vector Machine Modelling
The motivation behind support vector machines is that we are building a line of best fit between two datasets, where "best" is defined by an objective function of distance between our line of best fit and between critical points, called support vectors, of these datasets. Support vectors are the closest points to a line of best fit. Our best fit line is also a decision boundary.

In [16]:
from sklearn import svm

### Tests comparing the three kernels

In [15]:
svm_basic_tests = [
    Test(
        name='SVM linear',
        X=X,
        y=y,
        algorithm=svm.SVC,
        args={'probability': False, 'kernel': 'linear', 'C': 100}
    )
]

In [17]:
for test in svm_basic_tests:
    print(f'\n\n{test.name}\n' + '='*len(test.name))
    acc_train, acc_test = test.run()
    print(f'Training accuracy: {acc_train}')
    print(f'Test accuracy: {acc_test}')

NameError: name 'svm_basic_tests' is not defined

### Tests comparing different degrees of the polynomial kernel

In [13]:
svm_poly_tests = [Test(name=f'SVM poly of degree {i}', X=X_norm, y=y_norm, algorithm=svm.SVC, args={'probability': False, 'kernel': 'poly', 'C': 0.001, 'degree': i}) for i in range(1,11)]

In [14]:
for test in svm_poly_tests:
    print(f'\n\n{test.name}\n' + '='*len(test.name))
    acc_train, acc_test = test.run()
    print(f'Training accuracy: {acc_train}')
    print(f'Test accuracy: {acc_test}')



SVM poly of degree 1
Training accuracy: 0.643
Test accuracy: 0.652


SVM poly of degree 2
Training accuracy: 0.6415
Test accuracy: 0.658


SVM poly of degree 3
Training accuracy: 0.632
Test accuracy: 0.696


SVM poly of degree 4
Training accuracy: 0.643
Test accuracy: 0.652


SVM poly of degree 5
Training accuracy: 0.6445
Test accuracy: 0.646


SVM poly of degree 6
Training accuracy: 0.6525
Test accuracy: 0.614


SVM poly of degree 7
Training accuracy: 0.643
Test accuracy: 0.652


SVM poly of degree 8
Training accuracy: 0.6475
Test accuracy: 0.634


SVM poly of degree 9
Training accuracy: 0.649
Test accuracy: 0.628


SVM poly of degree 10
Training accuracy: 0.644
Test accuracy: 0.648


### Linear Regression Modelling

In [16]:
from sklearn import linear_model

In [25]:


logreg_l1_tests = [Test(name=f'Logreg l1 penalty with C={i}', X=X_norm, y=y_norm, algorithm=linear_model.LogisticRegression, args={'penalty':'l1', 'solver':'saga', 'C':i}) for i in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]]

for test in logreg_l1_tests:
    print(f'\n\n{test.name}\n' + '='*len(test.name))
    acc_train, acc_test = test.run()
    print(f'Training accuracy: {acc_train}')
    print(f'Test accuracy: {acc_test}')



Logreg l1 penalty with C=0.0001
Training accuracy: 0.6535
Test accuracy: 0.61


Logreg l1 penalty with C=0.001
Training accuracy: 0.6515
Test accuracy: 0.618


Logreg l1 penalty with C=0.01
Training accuracy: 0.635
Test accuracy: 0.684


Logreg l1 penalty with C=0.1
Training accuracy: 0.65
Test accuracy: 0.624


Logreg l1 penalty with C=1
Training accuracy: 0.6475
Test accuracy: 0.634


Logreg l1 penalty with C=10
Training accuracy: 0.6435
Test accuracy: 0.65


Logreg l1 penalty with C=100
Training accuracy: 0.652
Test accuracy: 0.616


Logreg l1 penalty with C=1000
Training accuracy: 0.6485
Test accuracy: 0.63


In [26]:
logreg_l2_tests = [Test(name=f'Logreg l2 penalty with C={i}', X=X_norm, y=y_norm, algorithm=linear_model.LogisticRegression, args={'penalty':'l1', 'solver':'saga', 'C':i}) for i in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]]

for test in logreg_l2_tests:
    print(f'\n\n{test.name}\n' + '='*len(test.name))
    acc_train, acc_test = test.run()
    print(f'Training accuracy: {acc_train}')
    print(f'Test accuracy: {acc_test}')



Logreg l2 penalty with C=0.0001
Training accuracy: 0.641
Test accuracy: 0.66


Logreg l2 penalty with C=0.001
Training accuracy: 0.6455
Test accuracy: 0.642


Logreg l2 penalty with C=0.01
Training accuracy: 0.6495
Test accuracy: 0.626


Logreg l2 penalty with C=0.1
Training accuracy: 0.641
Test accuracy: 0.66


Logreg l2 penalty with C=1
Training accuracy: 0.6465
Test accuracy: 0.638


Logreg l2 penalty with C=10
Training accuracy: 0.6545
Test accuracy: 0.606


Logreg l2 penalty with C=100
Training accuracy: 0.6465
Test accuracy: 0.638


Logreg l2 penalty with C=1000
Training accuracy: 0.6505
Test accuracy: 0.622
